In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re
from bs4 import BeautifulSoup
import json
import requests
from IPython.display import Image
import os
import urllib.request
import glob
import cv2


#Request API function
def requestJSON(url):
    res = requests.get(url)
    if res.status_code != 200:
        print(res.text)
        raise ValueError("Bad Response")
    return res.json()

lst = ['lacoste','miu-miu','louis-vuitton','chanel','altuzarra','hermes']
count = 0
urls = []
names = []
paths = []
for i in lst:
    url = f'https://www.vogue.com/fashion-shows/fall-2020-ready-to-wear/{i}/slideshow/collection/print'

    data = requests.get(url).text
    soup = BeautifulSoup(data, 'html.parser')
    images = soup.select('.slide--image')

    for ind,image in enumerate(images):
        url_name = image['src']
        name = "OK_"+str(count).zfill(6)
        urls.append(url_name)
        names.append(name)
        count+= 1
        
        #Extract the pictures
        path_name = '../inputs/images/People_Yes_fullHD/{}.jpg'.format(name)
        paths.append(path_name)
        urllib.request.urlretrieve(url_name, path_name)

#Create dataframe
df = pd.DataFrame()
df["urls"]=urls
df["names"]=names
df["paths"]=paths
df = df.set_index("names")
df = df.drop_duplicates()

## NOW THAT IT IS WORKING. Create the function.
def extractVogue(designer,collection,urls,names,paths,count=0):
      
    url = f'https://www.vogue.com/fashion-shows/{collection}/{designer}/slideshow/collection/print'

    data = requests.get(url).text
    soup = BeautifulSoup(data, 'html.parser')
    images = soup.select('.slide--image')
    
    for image in images:
        url_name = image['src']
        name = "OK_"+str(count).zfill(6)
        urls.append(url_name)
        names.append(name)
        count+=1
        
        #Extract the pictures
        path_name = '../inputs/images/People_Yes_fullHD/{}.jpg'.format(name)
        paths.append(path_name)
        urllib.request.urlretrieve(url_name, path_name)
        print(f"Extracting pictures from vogue.com designer:{designer}, collection:{collection}...")

    #Create dataframe
    df = pd.DataFrame()
    df["names"]=names
    df["urls"]=urls
    df["paths"]=paths
    df = df.set_index("names")
    df = df.drop_duplicates()
    
    return df,urls,names,paths,count

#Adding some extra values
df,urls,names,paths,count = extractVogue("moises-nieto","madrid-spring-2020",urls,names,paths,count)
df,urls,names,paths,count = extractVogue("jacquemus","spring-2020-menswear",urls,names,paths,count)
df,urls,names,paths,count = extractVogue("ludovic-de-saint-sernin","spring-2020-menswear",urls,names,paths,count)
df,urls,names,paths,count = extractVogue("abasi-rosborough","spring-2020-menswear",urls,names,paths,count)
df,urls,names,paths,count = extractVogue("alyx","fall-2020-menswear",urls,names,paths,count)
df,urls,names,paths,count = extractVogue("todd-snyder","spring-2019-menswear",urls,names,paths,count)
df,urls,names,paths,count = extractVogue("berluti","spring-2017-menswear",urls,names,paths,count)



                                                        urls  \
names                                                          
OK_000000  https://assets.vogue.com/photos/5e5e9a3cc66797...   
OK_000001  https://assets.vogue.com/photos/5e5e99ef02f19a...   
OK_000002  https://assets.vogue.com/photos/5e5e9a61b8b636...   
OK_000003  https://assets.vogue.com/photos/5e5e9a1ea173d7...   
OK_000004  https://assets.vogue.com/photos/5e5e99f36dd294...   

                                                      paths  
names                                                        
OK_000000  ../inputs/images/People_Yes_fullHD/OK_000000.jpg  
OK_000001  ../inputs/images/People_Yes_fullHD/OK_000001.jpg  
OK_000002  ../inputs/images/People_Yes_fullHD/OK_000002.jpg  
OK_000003  ../inputs/images/People_Yes_fullHD/OK_000003.jpg  
OK_000004  ../inputs/images/People_Yes_fullHD/OK_000004.jpg  


KeyError: 'paths'

In [8]:
#Clean df to create the GT = OK
df["GT"]="OK"
display(df.head(5))
print("Shape OK_pictures = ",df.shape)

#Create the GT = KO
KO_list = glob.glob("../inputs/images/People_No/*.jpg")
dfno = pd.DataFrame()
dfno["paths"]=paths
dfno["urls"]= "COCO_Dataset"
dfno = dfno.reset_index()
dfno = dfno.rename(columns={"index":"order"})
dfno["names"] = dfno.order.apply(lambda x: "KO_"+str(x).zfill(6))
dfno = dfno.drop(columns=["order"])
dfno["GT"]= dfno.names.apply(lambda x: "KO")
dfno = dfno.set_index("names")

display(dfno.head(5))
print("Shape KO_pictures = ",dfno.shape)

,urls,paths,GT
names,,,
OK_000000,https://assets.vogue.com/photos/5e5e9a3cc66797...,../inputs/images/People_Yes_fullHD/OK_000000.jpg,OK
OK_000001,https://assets.vogue.com/photos/5e5e99ef02f19a...,../inputs/images/People_Yes_fullHD/OK_000001.jpg,OK
OK_000002,https://assets.vogue.com/photos/5e5e9a61b8b636...,../inputs/images/People_Yes_fullHD/OK_000002.jpg,OK
OK_000003,https://assets.vogue.com/photos/5e5e9a1ea173d7...,../inputs/images/People_Yes_fullHD/OK_000003.jpg,OK
OK_000004,https://assets.vogue.com/photos/5e5e99f36dd294...,../inputs/images/People_Yes_fullHD/OK_000004.jpg,OK


(595, 3)


,paths,urls,GT
names,,,
KO_000000,../inputs/images/People_Yes_fullHD/OK_000000.jpg,COCO_Dataset,KO
KO_000001,../inputs/images/People_Yes_fullHD/OK_000001.jpg,COCO_Dataset,KO
KO_000002,../inputs/images/People_Yes_fullHD/OK_000002.jpg,COCO_Dataset,KO
KO_000003,../inputs/images/People_Yes_fullHD/OK_000003.jpg,COCO_Dataset,KO
KO_000004,../inputs/images/People_Yes_fullHD/OK_000004.jpg,COCO_Dataset,KO
